<a href="https://colab.research.google.com/github/tim142/natural_language_processing/blob/master/lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Practical 1: Sentiment Detection of Movie Reviews
========================================



This practical concerns sentiment detection of movie reviews.
In [this file](https://gist.githubusercontent.com/bastings/d47423301cca214e3930061a5a75e177/raw/5113687382919e22b1f09ce71a8fecd1687a5760/reviews.json) (80MB) you will find 1000 positive and 1000 negative **movie reviews**.
Each review is a **document** and consists of one or more sentences.

To prepare yourself for this practical, you should
have a look at a few of these texts to understand the difficulties of
the task (how might one go about classifying the texts?); you will write
code that decides whether a random unseen movie review is positive or
negative.

Please make sure you have read the following paper:

>   Bo Pang, Lillian Lee, and Shivakumar Vaithyanathan
(2002). 
[Thumbs up? Sentiment Classification using Machine Learning
Techniques](https://dl.acm.org/citation.cfm?id=1118704). EMNLP.

Bo Pang et al. were the "inventors" of the movie review sentiment
classification task, and the above paper was one of the first papers on
the topic. The first version of your sentiment classifier will do
something similar to Bo Pang’s system. If you have questions about it,
we should resolve them in our first demonstrated practical.


**Advice**

Please read through the entire practical and familiarise
yourself with all requirements before you start coding or otherwise
solving the tasks. Writing clean and concise code can make the difference
between solving the assignment in a matter of hours, and taking days to
run all experiments.

**Environment**

All code should be written in **Python 3**. 
If you use Colab, check if you have that version with `Runtime -> Change runtime type` in the top menu.

> If you want to work in your own computer, then download this notebook through `File -> Download .ipynb`.
The easiest way to
install Python is through downloading
[Anaconda](https://www.anaconda.com/download). 
After installation, you can start the notebook by typing `jupyter notebook filename.ipynb`.
You can also use an IDE
such as [PyCharm](https://www.jetbrains.com/pycharm/download/) to make
coding and debugging easier. It is good practice to create a [virtual
environment](https://docs.python.org/3/tutorial/venv.html) for this
project, so that any Python packages don’t interfere with other
projects.

#### Learning Python 3

If you are new to Python 3, you may want to check out a few of these resources:
- https://learnxinyminutes.com/docs/python3/
- https://www.learnpython.org/
- https://docs.python.org/3/tutorial/

Loading the Data
-------------------------------------------------------------

In [0]:
# download sentiment lexicon
!wget https://gist.githubusercontent.com/bastings/d6f99dcb6c82231b94b013031356ba05/raw/f80a0281eba8621b122012c89c8b5e2200b39fd6/sent_lexicon
# download review data
!wget https://gist.githubusercontent.com/bastings/d47423301cca214e3930061a5a75e177/raw/5113687382919e22b1f09ce71a8fecd1687a5760/reviews.json

--2019-11-13 17:51:13--  https://gist.githubusercontent.com/bastings/d6f99dcb6c82231b94b013031356ba05/raw/f80a0281eba8621b122012c89c8b5e2200b39fd6/sent_lexicon
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662577 (647K) [text/plain]
Saving to: ‘sent_lexicon’

sent_lexicon        100%[===================>] 647.05K  --.-KB/s    in 0.05s   

2019-11-13 17:51:13 (12.5 MB/s) - ‘sent_lexicon’ saved [662577/662577]

--2019-11-13 17:51:14--  https://gist.githubusercontent.com/bastings/d47423301cca214e3930061a5a75e177/raw/5113687382919e22b1f09ce71a8fecd1687a5760/reviews.json
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151

In [0]:
import math
import os
import sys
from subprocess import call
from nltk import FreqDist
from nltk.util import ngrams
from nltk.stem.porter import PorterStemmer
import sklearn as sk
from google.colab import drive
import pickle
import json
from collections import Counter
import requests
import matplotlib.pyplot as plt
import numpy as np
import time
from decimal import Decimal
from scipy.special import comb
from numpy import genfromtxt

In [0]:
with open("reviews.json", mode="r", encoding="utf-8") as f:
  reviews = json.load(f)  

# FUNCTIONS DATA PREPROCESSING

In [0]:
def create_reviews_processed(reviews, pos, stemming, n_grams):
  reviews_processed = []
  stemmer = PorterStemmer()

  # CREATE VOCABULARY LIST
  for review in reviews:
    new_review = []
    for sentence in review["content"]:
      for token, pos_tag in sentence:
        if stemming == 1:
          token = stemmer.stem(token)
        if pos == 1:
          word = token.lower() + "_" + pos_tag.lower()
          new_review.append(word)

        else:
          word = token.lower()
          new_review.append(word)

    all_n_grams = []

    #CREATE NUMBER OF DESIRED NGRAMS
    n = 0
    while n != n_grams:
      all_n_grams.append(list(ngrams(new_review, n+1, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>')))
      n += 1
    
    #FLATTEN LIST OF NGRAM LISTS INTO LIST
    all_n_grams = [item for sublist in all_n_grams for item in sublist]
    
    #CONVERT TUPLES TO STR
    str_grams = []
    for n_gram in all_n_grams:
        str_grams.append(' '.join(n_gram))
        
    #ADD REVIEW TO LIST OF REVIEWS FOR BOW
    reviews_processed.append(str_grams)
    
  return reviews_processed

In [0]:
def create_X(reviews, progress_printing):
  # CREATE VOCABULARY LIST
  D = Counter()
  for review in reviews:
    for token in review:
      D[token.lower()] += 1
  vocab = list(D)

  # CREATE BAG OF WORDS MARTIX

  # Progress printing
  if progress_printing == "progress_printing":
    print("PROGRESS (PROCESSED REVIEWS / TOTAL REVIEWS)")
  
  # Set datatype to int16 in order to prevent large data files
  X = np.zeros((len(reviews), len(vocab)+1))
  X = X.astype(np.int16)
  
  for review_ind, review in enumerate(reviews):

    # progress printing
    if progress_printing == "progress_printing":
      if review_ind in list(range(100, 2000, 100)):
        print(review_ind, "/", len(reviews))

    # Count words for review
    for token in review:
      word = token.lower()
      word_ind = vocab.index(word)
      X[review_ind][word_ind] += 1 

  return(X)

In [0]:
def create_Y(reviews):
  # CREATE BAG OF WORDS MARTIX
  Y = np.zeros((len(reviews), 1))

  for review_ind, review in enumerate(reviews):
    if review["sentiment"] == "POS": 
      sentiment = 1
    else:
      sentiment = 0
    Y[review_ind][0] = sentiment

  return(Y)

In [0]:
# Save BOW als .npy file
def save_X(pos, stemming, n_grams, progress_printing):
  start_time = time.time()
  reviews_processed = create_reviews_processed(reviews, pos, stemming, n_grams) 
  X = create_X(reviews_processed, progress_printing)

  print("--- time: %s seconds ---" % round((time.time() - start_time), 2), "\n")

  file_name = "X{}{}{}.npy".format(pos, stemming, n_grams)
  np.save(file_name, X)

# GENERATING DATA

In [0]:
Y = create_Y(reviews)
np.save("Y", Y)

In [0]:
save_X(0, 0, 1, "no_progress_printing")

--- time: 60.92 seconds --- 



In [0]:
save_X(0, 1, 1, "no_progress_printing")

--- time: 66.78 seconds --- 



In [0]:
save_X(1, 0, 1, "no_progress_printing")

--- time: 76.8 seconds --- 



In [0]:
save_X(0, 0, 3, "progress_printing") # pos=0, stemming=0, n_grams=3

PROGRESS (PROCESSED REVIEWS / TOTAL REVIEWS)
100 / 2000
200 / 2000
300 / 2000
400 / 2000
500 / 2000
600 / 2000
700 / 2000
800 / 2000
900 / 2000
1000 / 2000
1100 / 2000
1200 / 2000
1300 / 2000
1400 / 2000
1500 / 2000
1600 / 2000
1700 / 2000
1800 / 2000
1900 / 2000


Symbolic approach – sentiment lexicon (2pts)
---------------------------------------------------------------------



**How** could one automatically classify movie reviews according to their
sentiment? 

If we had access to a **sentiment lexicon**, then there are ways to solve
the problem without using Machine Learning. One might simply look up
every open-class word in the lexicon, and compute a binary score
$S_{binary}$ by counting how many words match either a positive, or a
negative word entry in the sentiment lexicon $SLex$.

$$S_{binary}(w_1w_2...w_n) = \sum_{i = 1}^{n}\text{sgn}(SLex\big[w_i\big])$$

**Threshold.** In average there are more positive than negative words per review (~7.13 more positive than negative per review) to take this bias into account you should use a threshold of **8** (roughly the bias itself) to make it harder to classify as positive.

$$
\text{classify}(S_{binary}(w_1w_2...w_n)) = \bigg\{\begin{array}{ll}
        \text{positive} & \text{if } S_{binary}(w_1w_2...w_n) > threshold\\
        \text{negative} & \text{else }
        \end{array}
$$

To implement this approach, you should use the sentiment
lexicon in `sent_lexicon`, which was taken from the
following work:

> Theresa Wilson, Janyce Wiebe, and Paul Hoffmann
(2005). [Recognizing Contextual Polarity in Phrase-Level Sentiment
Analysis](http://www.aclweb.org/anthology/H/H05/H05-1044.pdf). HLT-EMNLP.

#### (Q: 1.1) Implement this approach and report its classification accuracy. (1 pt)

In [0]:
import csv
with open('sent_lexicon') as csvfile:
    sent_lexicon = csv.reader(csvfile, delimiter=' ')

    # first, we construct a dictionary where each key is a word, and each value is the sentiment and its magnitude
    sentiment_dic = {}

    for entry in sent_lexicon:
        if entry[-1] == 'priorpolarity=negative' and entry[0] == 'type=strongsubj':
            sentiment_dic[entry[2][6:]] = 'strong negative'
        elif entry[-1] == 'priorpolarity=negative' and entry[0] == 'type=weaksubj':
            sentiment_dic[entry[2][6:]] = 'weak negative'
        elif entry[-1] == 'priorpolarity=positive'and entry[0] == 'type=strongsubj':
            sentiment_dic[entry[2][6:]] = 'strong positive'
        elif entry[-1] == 'priorpolarity=positive'and entry[0] == 'type=weaksubj':
            sentiment_dic[entry[2][6:]] = 'weak positive'

def symbolic_classifyer_1(corpus):
    token_results = []
    correct = 0
    incorrect = 0

    for review in corpus:
        threshold = 8
        pos_words = 0
        neg_words = 0

        #iterate over words in review, count all positive and negative words and store in counter variables
        for sentence in review["content"]:
            for token, pos_tag in sentence:
                token = token.lower()
                try:
                    if sentiment_dic[token]:
                        if sentiment_dic[token] == 'strong positive':
                            pos_words += 1
                        elif sentiment_dic[token] == 'weak positive':
                            pos_words += 1
                        elif sentiment_dic[token] == 'strong negative':
                            neg_words += 1
                        elif sentiment_dic[token] == 'weak negative':
                            neg_words += 1
                except:
                    continue          

        #determine score for each review            
        if (pos_words - threshold) > neg_words:
            sentiment = 'POS'
        else:
            sentiment = 'NEG'

        #check    
        if sentiment == review['sentiment']:
            correct += 1
            token_results.append(1)
        else:
            incorrect += 1
            token_results.append(0)

    total_predictions = correct + incorrect
    accuracy = (correct / total_predictions)
    print("Accuracy = {}".format(accuracy))
    return token_results

In [0]:
results_Q11 = symbolic_classifyer_1(reviews)

Accuracy = 0.6785


Results

If the sentiment lexicon also has information about the **magnitude** of
sentiment (e.g., *“excellent"* would have higher magnitude than
*“good"*), we could take a more fine-grained approach by adding up all
sentiment scores, and deciding the polarity of the movie review using
the sign of the weighted score $S_{weighted}$.

$$S_{weighted}(w_1w_2...w_n) = \sum_{i = 1}^{n}SLex\big[w_i\big]$$


Their lexicon also records two possible magnitudes of sentiment (*weak*
and *strong*), so you can implement both the binary and the weighted
solutions (please use a switch in your program). For the weighted
solution, you can choose the weights intuitively *once* before running
the experiment.

#### (Q: 1.2) Now incorporate magnitude information and report the classification accuracy. Don't forget to use the threshold. (1 pt)

In [0]:
def symbolic_classifyer_2(corpus, switch):
    
    #list of classification results. 
    magnitude_results = []
    
    #define weights for sentiment magnitude
    pos_strong = 2
    pos_weak = 1
    neg_strong = 2
    neg_weak = 1
    
    #switch
    if switch == 0:
        pos_strong = 1
        neg_strong = 1
    else:
        pos_strong = 2
        neg_strong = 2 
    
    #counter variables for correct and incorrect classifications
    correct = 0
    incorrect = 0

    for review in corpus:
        threshold = 8
        pos_words = 0
        neg_words = 0

        #iterate over words in review, count all positive and negative words and store in counter variables
        for sentence in review["content"]:
            for token, pos_tag in sentence:
                token = token.lower()
                try:
                    if sentiment_dic[token]:
                        if sentiment_dic[token] == 'strong positive':
                            pos_words += pos_strong
                        elif sentiment_dic[token] == 'weak positive':
                            pos_words += pos_weak
                        elif sentiment_dic[token] == 'strong negative':
                            neg_words += neg_strong
                        elif sentiment_dic[token] == 'weak negative':
                            neg_words += neg_weak
                except:
                    continue
                
        #determine score for each review            
        if (pos_words - threshold) > neg_words:
            sentiment = 'POS'
        else:
            sentiment = 'NEG'

        #check    
        if sentiment == review['sentiment']:
            correct += 1
            magnitude_results.append(1)
        else:
            incorrect += 1
            magnitude_results.append(0)

    total_predictions = correct + incorrect
    accuracy = (correct / total_predictions)
    print("Accuracy = {}".format(accuracy))
    return magnitude_results

Results

In [0]:
results_Q12 = symbolic_classifyer_2(reviews,1)

Accuracy = 0.6865


#### Optional: make a barplot of the two results.

In [0]:
# YOUR CODE HERE

Answering questions in statistically significant ways (1pt)
-------------------------------------------------------------

Does using the magnitude improve the results? Oftentimes, answering questions like this about the performance of
different signals and/or algorithms by simply looking at the output
numbers is not enough. When dealing with natural language or human
ratings, it’s safe to assume that there are infinitely many possible
instances that could be used for training and testing, of which the ones
we actually train and test on are a tiny sample. Thus, it is possible
that observed differences in the reported performance are really just
noise. 

There exist statistical methods which can be used to check for
consistency (*statistical significance*) in the results, and one of the
simplest such tests is the **sign test**. 

The sign test is based on the binomial distribution. Count all cases when System 1 is better than System 2, when System 2 is better than System 1, and when they are the same. Call these numbers $Plus$, $Minus$ and $Null$ respectively. 

The sign test returns the probability that the null hypothesis is true. 

This probability is called the $p$-value and it can be calculated for the two-sided sign test using the following formula (we multiply by two because this is a two-sided sign test and tests for the significance of differences in either direction):

$$2 \, \sum\limits_{i=0}^{k} \binom{N}{i} \, q^i \, (1-q)^{N-i}$$

where $$N = 2 \Big\lceil \frac{Null}{2}\Big\rceil + Plus + Minus$$ is the total
number of cases, and
$$k = \Big\lceil \frac{Null}{2}\Big\rceil + \min\{Plus,Minus\}$$ is the number of
cases with the less common sign. 

In this experiment, $q = 0.5$. Here, we
treat ties by adding half a point to either side, rounding up to the
nearest integer if necessary. 


#### (Q 2.1): Implement the sign test. Is the difference between the two symbolic systems significant? What is the p-value? (1 pt)

You should use the `comb` function from `scipy` and the `decimal` package for the stable adding of numbers in the final summation.

You can quickly verify the correctness of
your sign test code using a [free online
tool](https://www.graphpad.com/quickcalcs/binomial1.cfm).

In [0]:
def sign_test(results_1, results_2):
  """test for significance
  results_1 is a list of classification results (+ for correct, - incorrect)
  results_2 is a list of classification results (+ for correct, - incorrect)
  """
  ties, plus, minus = 0, 0, 0

  # "-" carries the error
  for i in range(0, len(results_1)):
    if results_1[i]==results_2[i]:
      ties += 1
    elif results_1[i]==0: 
      plus += 1
    elif results_2[i]==0: 
      minus += 1

  n = 2 * (ties / 2) + plus + minus
  k = (ties / 2) + min(plus, minus)

  summation = Decimal(0.0)
  for i in range(0,int(k)+1):
      summation += (comb(n,i, exact=True))

  # use two-tailed version of test
  summation *= 2
  summation *= (Decimal(0.5)**Decimal(n))
  print("P-value = ", summation)
  print("the difference is", 
        "not significant" if summation >= 0.05 else "significant")
  return summation

Checking for significance between the two symbolic systems of Question 1.1 and 1.2:


In [0]:
p_value = sign_test(results_Q11, results_Q12)

P-value =  0.7373250398720036864195787185
the difference is not significant


## Using the Sign test

**From now on, report all differences between systems using the
sign test.** You can think about a change that you apply to one system, as a
 new system.
    
You should report statistical test
results in an appropriate form – if there are several different methods
(i.e., systems) to compare, tests can only be applied to pairs of them
at a time. This creates a triangular matrix of test results in the
general case. When reporting these pair-wise differences, you should
summarise trends to avoid redundancy.


Naive Bayes (8pt + 1pt bonus)
==========


Your second task is to program a simple Machine Learning approach that operates
on a simple Bag-of-Words (BoW) representation of the text data, as
described in Pang et al. (2002). In this approach, the only features we
will consider are the words in the text themselves, without bringing in
external sources of information. The BoW model is a popular way of
representing text information as vectors (or points in space), making it
easy to apply classical Machine Learning algorithms on NLP tasks.
However, the BoW representation is also very crude, since it discards
all information related to word order and grammatical structure in the
original text.

## Writing your own classifier

Write your own code to implement the Naive Bayes (NB) classifier. As
a reminder, the Naive Bayes classifier works according to the following
equation:
$$\hat{c} = \operatorname*{arg\,max}_{c \in C} P(c|\bar{f}) = \operatorname*{arg\,max}_{c \in C} P(c)\prod^n_{i=1} P(f_i|c)$$
where $C = \{ \text{POS}, \text{NEG} \}$ is the set of possible classes,
$\hat{c} \in C$ is the most probable class, and $\bar{f}$ is the feature
vector. Remember that we use the log of these probabilities when making
a prediction:
$$\hat{c} = \operatorname*{arg\,max}_{c \in C} \Big\{\log P(c) + \sum^n_{i=1} \log P(f_i|c)\Big\}$$

You can find more details about Naive Bayes in [Jurafsky &
Martin](https://web.stanford.edu/~jurafsky/slp3/). You can also look at
this helpful
[pseudo-code](https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html).

*Note: this section and the next aim to put you a position to replicate
    Pang et al., Naive Bayes results. However, the numerical results
    will differ from theirs, as they used different data.*

**You must write the Naive Bayes training and prediction code from
scratch.** You will not be given credit for using off-the-shelf Machine
Learning libraries.

The data contains the text of the reviews, where each document consists
of the sentences in the review, the sentiment of the review and an index
(cv) that you will later use for cross-validation. You will find the
text has already been tokenised and POS-tagged for you. Your algorithm
should read in the text, **lowercase it**, and store the words and their
frequencies in an appropriate data structure that allows for easy
computation of the probabilities used in the Naive Bayes algorithm, and
then make predictions for new instances.

#### (Q3.1) Train your classifier on (positive and negative) reviews with cv-value 000-899, and test it on the remaining reviews cv900–cv999.  Report results using simple classification accuracy as your evaluation metric. Your  features are the word vocabulary. The value of a feature is the count of that feature (word) in the document. (2pts)


In [0]:
def train_multinomial_NB(X, Y, smoothing):
  # CLASS PRIORS [NEG, POS]
  classes = np.unique(Y)
  Y_summed = np.sum(Y, axis=0)
  priors = np.array([[1-Y_summed[0]/len(Y), Y_summed[0]/len(Y)]]).T
  
  # CLASS CONDITIONAL PROBABILITIES [NEG, POS]
  conditionals = np.zeros(len(X[0]))

  for i, c in enumerate(classes): 
    c_ind = np.where(Y == c)
    X_class = X[c_ind[0], :]
    X_class_summed = np.sum(X_class, axis=0) 
    
    if smoothing == "laplace":
      conditional = (X_class_summed + 1) / (sum(X_class_summed) + len(X[0]))
    else:
      conditional = (X_class_summed) / sum(X_class_summed)
    
    conditionals = np.vstack((conditionals, conditional))

  conditionals = conditionals[1:]

  return(priors, conditionals)

In [0]:
def apply_multinomial_NB(X, priors, conditionals):
  posterior_neg = np.dot(X, np.log(conditionals[0])) + np.log(priors[0])
  posterior_pos = np.dot(X, np.log(conditionals[1])) + np.log(priors[1])

  merged = np.vstack((posterior_neg, posterior_pos,))
  predictions = np.argmax(merged, axis=0)
  return(predictions)

In [0]:
# Load input and out data
X = np.load('X001.npy') # settings: pos=0, stemming=0, n_grams=1
Y = np.load('Y.npy')

# Training data
X_TRAIN = np.vstack((X[0:900], X[1000:1900]))
Y_TRAIN = np.vstack((Y[0:900], Y[1000:1900]))

# Validation data
X_VALID = np.vstack((X[900:1000], X[1900:2000]))
Y_VALID = np.vstack((Y[900:1000], Y[1900:2000]))
Y_VALID = Y_VALID.T[0].astype(int)

# Shapes
print("shape X_TRAIN: ", np.shape(X_TRAIN))
print("shape Y_TRAIN: ", np.shape(Y_TRAIN))
print("shape X_VALID: ", np.shape(X_VALID))
print("shape Y_VALID: ", np.shape(Y_VALID))

shape X_TRAIN:  (1800, 47744)
shape Y_TRAIN:  (1800, 1)
shape X_VALID:  (200, 47744)
shape Y_VALID:  (200,)


In [0]:
# Training
priors, conditionals = train_multinomial_NB(X_TRAIN, Y_TRAIN, 0)

# Classification
predictions = apply_multinomial_NB(X_VALID, priors, conditionals)

# Evaluation
results = abs((abs(predictions - Y_VALID)) - 1) 
accuracy = sum(results)/len(results)
np.save('Q31_results_0010.npy', results)  

print("results: ", results, "\n")
print("accuracy: ", accuracy)

print("\nThe low accuracy is the result of not applying smoothing,")
print("due to which the algorithm encounters divisions by zero.\n\n")

results:  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] 

accuracy:  0.5

The low accuracy is the result of not applying smoothing,
due to which the algorithm encounters divisions by zero.




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


***Naive Bayes without smoothing results in an accuracy of 0.5***

#### (Bonus Questions) Would you consider accuracy to also be a good way to evaluate your classifier in a situation where 90% of your data instances are of positive movie reviews? (1pt)

You can simulate this scenario by keeping the positive reviews
data unchanged, but only using negative reviews cv000–cv089 for
training, and cv900–cv909 for testing. Calculate the classification
accuracy, and explain what changed.

***In the case of imbalanced data, that is, the vast majority being either positive or negative, accuracy might not be an optimal evaluation method. Say, 90% of the data is positive, the algorithm could just label every entry as positive and thus achieve an accuracy of 90% without actually doing any learning. To account for this, supplementing model evaluation with precision and recall metrics would be appropriate.***

## Smoothing

The presence of words in the test dataset that
haven’t been seen during training can cause probabilities in the Naive
Bayes classifier to be $0$, thus making that particular test instance
undecidable. The standard way to mitigate this effect (as well as to
give more clout to rare words) is to use smoothing, in which the
probability fraction
$$\frac{\text{count}(w_i, c)}{\sum\limits_{w\in V} \text{count}(w, c)}$$ for a word
$w_i$ becomes
$$\frac{\text{count}(w_i, c) + \text{smoothing}(w_i)}{\sum\limits_{w\in V} \text{count}(w, c) + \sum\limits_{w \in V} \text{smoothing}(w)}$$





#### (Q3.2) Implement Laplace feature smoothing (1pt)
($smoothing(\cdot) = \kappa$, constant for all words) in your Naive
Bayes classifier’s code, and report the impact on performance. 
Use $\kappa = 1$.

In [0]:
# Training
priors, conditionals = train_multinomial_NB(X_TRAIN, Y_TRAIN, "laplace")

# Classification
predictions = apply_multinomial_NB(X_VALID, priors, conditionals)

# Evaluation
results = abs((abs(predictions - Y_VALID)) - 1) 
accuracy = sum(results)/len(results)
np.save('Q32_results_001L.npy', results)  

print("results: ", results, "\n")
print("accuracy: ", accuracy)

results:  [1 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1
 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1
 0 1 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0
 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 0 1 0 1 0 0 1 0 1 1 1 1 1] 

accuracy:  0.825


***Naive Bayes with smoothing results in an accuracy of 0.825. However, whether this result is significantly better than the Naive Bayes classififer without smoothing or whether the difference in accuracy is random, cannot be said. In order to determine whether the difference is significant, we will apply the significance test below.***

#### (Q3.3) Is the difference between non smoothed (Q3.1) and smoothed (Q3.2) statistically significant? (0.5pt)

In [0]:
# Without smoothing
Q31_results_0010 = np.load('Q31_results_0010.npy')

# With laplace smoothing
Q32_results_001L = np.load('Q32_results_001L.npy')

p_value = sign_test(Q31_results_0010, Q32_results_001L)

P-value = 0.0000
the difference is significant
p_value =  0.000003547178174130642586494974890


***P value = 0.0000035, so the difference is significant***

## Cross-validation

A serious danger in using Machine Learning on small datasets, with many
iterations of slightly different versions of the algorithms, is that we
end up with Type III errors, also called the “testing hypotheses
suggested by the data” errors. This type of error occurs when we make
repeated improvements to our classifiers by playing with features and
their processing, but we don’t get a fresh, never-before seen test
dataset every time. Thus, we risk developing a classifier that’s better
and better on our data, but worse and worse at generalizing to new,
never-before seen data.

A simple method to guard against Type III errors is to use
cross-validation. In N-fold cross-validation, we divide the data into N
distinct chunks / folds. Then, we repeat the experiment N times, each
time holding out one of the chunks for testing, training our classifier
on the remaining N - 1 data chunks, and reporting performance on the
held-out chunk. We can use different strategies for dividing the data:

-   Consecutive splitting:
  - cv000–cv099 = Split 1
  - cv100–cv199 = Split 2
  - etc.
  
-   Round-robin splitting (mod 10):
  - cv000, cv010, cv020, … = Split 1
  - cv001, cv011, cv021, … = Split 2
  - etc.

-   Random sampling/splitting
  - Not used here (but you may choose to split this way in a non-educational situation)

#### (Q3.4) Write the code to implement 10-fold cross-validation using round-robin splitting for your Naive Bayes classifier from Q3.2 and compute the 10 accuracies. Report the final performance, which is the average of the performances per fold. If all splits perform equally well, this is a good sign. (1pt)






In [0]:
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification

def cross_validate(model, X, Y, folds):
    results = Y[0:len(Y):10].T[0].astype(int)
    results = np.zeros(np.shape(results))
    accuracies = []
    
    for i in range(folds):
        print("FOLD ", i+1, "/ 10")
        #CREATE VALIDATION SET ACCORDING TO NUMBER OF CHUNKS
        X_VALID = X[i:len(X):folds]
        Y_VALID = Y[i:len(Y):folds]
        Y_VALID = Y_VALID.T[0].astype(int)

        #CREATE TRAINING SETS
        X_TRAIN = np.delete(X, slice(i, len(X), folds), axis = 0)
        Y_TRAIN = np.delete(Y, slice(i, len(Y), folds), axis = 0)

        # TRAIN AND APPLY MODEL 
        if model == 'NB':
            priors, conditionals = train_multinomial_NB(X_TRAIN, Y_TRAIN, "laplace")
            predictions = apply_multinomial_NB(X_VALID, priors, conditionals)
        elif model == 'SVM':
            Y_TRAIN = Y_TRAIN.ravel()
            clf = LinearSVC(random_state=0, tol=1e-5, max_iter = 20000)
            clf.fit(X_TRAIN, Y_TRAIN)  
            predictions = clf.predict(X_VALID)

        # UPDATE RESULTS 
        fold_results = abs((abs(predictions - Y_VALID)) - 1)
        results = np.vstack((results, fold_results))
        
        # UPDATE ACCURACIES
        right = sum(fold_results)
        total = len(fold_results)
        accuracy = right/total
        accuracies.append(accuracy)

    results = results[1:].flatten()
    variance = np.var(accuracies)
    return(results, accuracies, variance)

In [0]:
# Load input and output data
X = np.load('X001.npy') # settings: pos=0, stemming=0, n_grams=1
Y = np.load('Y.npy')

model = "NB"
folds = 10

results, accuracies, variance = cross_validate(model, X, Y, folds)
np.save('Q34_CV_results_001L', results)

av_accuracy = sum(accuracies)/folds
print("accuracies: ", accuracies)
print("average_accuracy: ", av_accuracy)

FOLD  1 / 10
FOLD  2 / 10
FOLD  3 / 10
FOLD  4 / 10
FOLD  5 / 10
FOLD  6 / 10
FOLD  7 / 10
FOLD  8 / 10
FOLD  9 / 10
FOLD  10 / 10
accuracies:  [0.79, 0.835, 0.81, 0.83, 0.775, 0.845, 0.83, 0.785, 0.825, 0.845]
average_accuracy:  0.817


***Doing cross validation on the Naive Bayes classifier using smoothing results in an average accuracy of 0.817,based on the fold accuracies [0.79, 0.835, 0.81, 0.83, 0.775, 0.845, 0.83, 0.785, 0.825, 0.845]***

#### (Q3.5) Write code to calculate and report variance, in addition to the final performance. (1pt)

Please report all future results using 10-fold cross-validation now
(unless told to use the held-out test set).

In [0]:
print("variance: ", variance)

variance:  0.0005859999999999986


***The variance over the accuracies iss 0.000586, so all splits perform approximately the same.***

## Features, overfitting, and the curse of dimensionality

In the Bag-of-Words model, ideally we would like each distinct word in
the text to be mapped to its own dimension in the output vector
representation. However, real world text is messy, and we need to decide
on what we consider to be a word. For example, is “`word`" different
from “`Word`", from “`word`”, or from “`words`"? Too strict a
definition, and the number of features explodes, while our algorithm
fails to learn anything generalisable. Too lax, and we risk destroying
our learning signal. In the following section, you will learn about
confronting the feature sparsity and the overfitting problems as they
occur in NLP classification tasks.

#### (Q3.6): A touch of linguistics (1pt)

Taking a step further, you can use stemming to
hash different inflections of a word to the same feature in the BoW
vector space. How does the performance of your classifier change when
you use stemming on your training and test datasets? Please use the [Porter stemming
    algorithm](http://www.nltk.org/howto/stem.html) from NLTK.
 Also, you should do cross validation and concatenate the predictions from all folds to compute the significance.

In [0]:
# Load input and output data
X = np.load('X011.npy') # settings: pos=0, stemming=1, n_grams=1
Y = np.load('Y.npy')

model = "NB"
folds = 10

results, accuracies, variance = cross_validate(model, X, Y, folds)
np.save('Q36_CV_results_011L', results)

av_accuracy = sum(accuracies)/folds
print("accuracies: ", accuracies)
print("average_accuracy: ", av_accuracy)
print("variance: ", variance)

accuracies:  [0.78, 0.84, 0.81, 0.85, 0.775, 0.835, 0.815, 0.775, 0.83, 0.84]
average_accuracy:  0.8150000000000001


***Applying the Naive Bayes algorithm on the data with stemming results in an accuracy of 0.815***

#### (Q3.7): Is the difference between NB with smoothing and NB with smoothing+stemming significant? (0.5pt)


In [0]:
# Without stemming
Q34_CV_results_001L = np.load('Q34_CV_results_001L.npy')

# With stemming
Q36_CV_results_011L = np.load('Q36_CV_results_011L.npy')

p_value = sign_test(Q34_CV_results_001L, Q36_CV_results_011L)

P-value = 0.9465
the difference is not significant
p_value =  0.9465186089423488346539016590


***The p-value is 0.95, so the difference is not significant.***

#### Q3.8: What happens to the number of features (i.e., the size of the vocabulary) when using stemming as opposed to (Q3.2)? (0.5pt)
Give actual numbers. You can use the held-out training set to determine these.

In [0]:
X = np.load('X001.npy')
print("Features without stemmming: ", np.size(X[0]))

X = np.load('X011.npy')
print("Features with stemmming: ", np.size(X[0]))

Features without stemmming:  47744
Features with stemmming:  34201


***The number of features decreases when using stemming because multiple words could share the same stem, and thus merge into a single feature instead of each having their individual feature. ***

***Features without stemming: 47744***

***Features with stemming: 34201***




#### Q3.9: Putting some word order back in (0.5+0.5pt=1pt)

A simple way of retaining some of the word
order information when using bag-of-words representations is to add **n-grams** features. 
Retrain your classifier from (Q3.4) using **unigrams+bigrams** and
**unigrams+bigrams+trigrams** as features, and report accuracy and statistical significances (in comparison to the experiment at (Q3.4) for all 10 folds, and between the new systems).





In [0]:
model = "NB"
X = np.load('X002.npy')
Y = np.load('Y.npy')
folds = 10

results, accuracies, variance = cross_validate(model, X, Y, folds)
np.save('Q39_CV_results_002L', results)

av_accuracy = sum(accuracies)/folds
print("average_accuracy: ", av_accuracy)
print("variance: ", variance)

***Applying Naive Bayes without stemming and with using bigrams results in an accuracy of 0.842***

In [0]:
model = "NB"
X = np.load('X003.npy')
Y = np.load('Y.npy')
folds = 10

results, accuracies, variance = cross_validate(model, X, Y, folds)
np.save('Q39_CV_results_003L', results)

av_accuracy = sum(accuracies)/folds
print("average_accuracy: ", av_accuracy)
print("variance: ", variance)

***Applying Naive Bayes without stemming and with using bigrams results in an accuracy of 0.85***

In [0]:
# unigrams
Q34_CV_results_001L = np.load('Q34_CV_results_001L.npy')

# bigrams
Q39_CV_results_002L = np.load('Q39_CV_results_002L.npy')

# trigrams
Q39_CV_results_003L = np.load('Q39_CV_results_003L.npy')

p_value = sign_test(Q39_CV_results_001L, Q39_CV_results_002L)
p_value = sign_test(Q39_CV_results_001L, Q39_CV_results_003L)
p_value = sign_test(Q39_CV_results_002L, Q39_CV_results_003L)

**significance test for unigrams and bigrams**

**P-value = 0.2541**

**the difference is not significant**

**---------------------------------------------**

**significance test for unigrams and trigrams**

**P-value = 0.1461**

**the difference is not significant**

**---------------------------------------------**

**significance test for bigrams and trigrams**

**P-value = 0.7373**

**the difference is not significant**



#### Q3.10: How many features does the BoW model have to take into account now? (0.5pt)
How does this number compare (e.g., linear, square, cubed, exponential) to the number of features at (Q3.8)? 

Use the held-out training set once again for this.


In [0]:
X = np.load('X001.npy')
print("Features without stemmming: ", np.size(X[0]))

X = np.load('X011.npy')
print("Features with stemming): ", np.size(X[0]))

X = np.load('X002.npy')
print("Features with n=2 (no stemming): ", np.size(X[0]))

X = np.load('X003.npy')
print("Features with n=3 (no stemming): ", np.size(X[0]))

***Number of features without stemmming:  47744***



***Number of features with stemming:  34201***



***Number of features with unigrams+bigrams, no stemming:  508580***



***Number of features with unigrams+bigrams+trigrams, no stemming:  1551076***

***-------------------------------------------***


***It appears that the BoW feature space increases exponentially for larger values of n: number of features = ~ V^n***


# Support Vector Machines (4pts)


Though simple to understand, implement, and debug, one
major problem with the Naive Bayes classifier is that its performance
deteriorates (becomes skewed) when it is being used with features which
are not independent (i.e., are correlated). Another popular classifier
that doesn’t scale as well to big data, and is not as simple to debug as
Naive Bayes, but that doesn’t assume feature independence is the Support
Vector Machine (SVM) classifier.

You can find more details about SVMs in Chapter 7 of Bishop: Pattern Recognition and Machine Learning.
Other sources for learning SVM:
* http://web.mit.edu/zoya/www/SVM.pdf
* http://www.cs.columbia.edu/~kathy/cs4701/documents/jason_svm_tutorial.pdf
* https://pythonprogramming.net/support-vector-machine-intro-machine-learning-tutorial/







Use the scikit-learn implementation of 
[SVM.](http://scikit-learn.org/stable/modules/svm.html) with the default parameters.



#### (Q4.1): Train SVM and compare to Naive Bayes (2pt)

Train an SVM classifier (sklearn.svm.LinearSVC) using your features. Compare the
classification performance of the SVM classifier to that of the Naive
Bayes classifier from (Q3.4) and report the numbers.
Do cross validation and concatenate the predictions from all folds to compute the significance.  Are the results significantly better?



In [0]:
# Load input and output data
X = np.load('X001.npy')
Y = np.load('Y.npy')

model = "SVM"
folds = 10

results, accuracies, variance = cross_validate(model, X, Y, folds)
np.save('Q41_CV_results_001L', results)

av_accuracy = sum(accuracies)/folds
print("accuracies: ", accuracies)
print("average_accuracy: ", av_accuracy)
print("variance: ", variance)

FOLD  1 / 10
FOLD  2 / 10
FOLD  3 / 10
FOLD  4 / 10
FOLD  5 / 10
FOLD  6 / 10
FOLD  7 / 10
FOLD  8 / 10
FOLD  9 / 10
FOLD  10 / 10
accuracies:  [0.81, 0.795, 0.8, 0.84, 0.85, 0.815, 0.845, 0.85, 0.875, 0.84]
average_accuracy:  0.8320000000000001
variance:  0.0005959999999999991


**Applying an SVM without stemming and using unigrams results in an accuracy of 0.832**

In [0]:
# Without stemming
Q34_CV_results_001L = np.load('Q34_CV_results_001L.npy')

# With stemming
Q41_CV_results_001L = np.load('Q41_CV_results_001L.npy')

p_value = sign_test(Q34_CV_results_001L, Q41_CV_results_001L)

P-value =  0.5166977962573607869966679093
the difference is not significant


***The p-value is 0.52, so the difference between the SVM classifier and the NB classifier is not significant.***

### More linguistics

Now add in part-of-speech features. You will find the
movie review dataset has already been POS-tagged for you. Try to
replicate what Pang et al. were doing:



####(Q4.2) Replace your features with word+POS features, and report performance with the SVM. Does this help? Do cross validation and concatenate the predictions from all folds to compute the significance. Are the results significant? Why?  (1pt)


In [0]:
# Load input and output data
X = np.load('X101.npy') 
Y = np.load('Y.npy')

model = "SVM"
folds = 10

results, accuracies, variance = cross_validate(model, X, Y, folds)
np.save('Q42_CV_results_101L', results)

av_accuracy = sum(accuracies)/folds
print("accuracies: ", accuracies)
print("average_accuracy: ", av_accuracy)
print("variance: ", variance)

FOLD  1 / 10
FOLD  2 / 10
FOLD  3 / 10
FOLD  4 / 10
FOLD  5 / 10
FOLD  6 / 10
FOLD  7 / 10
FOLD  8 / 10
FOLD  9 / 10
FOLD  10 / 10
accuracies:  [0.82, 0.795, 0.825, 0.84, 0.84, 0.845, 0.855, 0.855, 0.865, 0.84]
average_accuracy:  0.8380000000000001
variance:  0.0003709999999999997


**Applying an SVM without stemming and using unigrams and POS tags results in an accuracy of 0.838**

In [0]:
# Without stemming
Q41_CV_results_001L = np.load('Q41_CV_results_001L.npy')

# With stemming
Q42_CV_results_101L = np.load('Q42_CV_results_101L.npy')

p_value = sign_test(Q41_CV_results_001L, Q42_CV_results_101L)

P-value =  0.8057148676803825624105038557
the difference is not significant


***The p-value is 0.806, so the difference between the SVM with and without using POS_tags is not significant.***

#### (Q4.3) Discard all closed-class words from your data (keep only nouns (N*), verbs (V*), adjectives (J*) and adverbs (RB*)), and report performance. Does this help? Do cross validation and concatenate the predictions from all folds to compute the significance. Are the results significantly better than when we don't discard the closed-class words? Why? (1pt)

In [0]:
def create_reviews_processed_Q43(reviews, pos, stemming, n_grams):
  reviews_processed = []
  stemmer = PorterStemmer()

  # CREATE VOCABULARY LIST
  for review in reviews:
    new_review = []
    for sentence in review["content"]:
      for token, pos_tag in sentence:
        if (pos_tag != "N") and (pos_tag != "V") and (pos_tag != "J") and (pos_tag !="RB"):
          if stemming == 1:
            token = stemmer.stem(token)
          if pos == 1:
            word = token.lower() + "_" + pos_tag.lower()
            new_review.append(word)

          else:
            word = token.lower()
            new_review.append(word)

    all_n_grams = []

    #CREATE NUMBER OF DESIRED NGRAMS
    n = 0
    while n != n_grams:
      all_n_grams.append(list(ngrams(new_review, n+1, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>')))
      n += 1
    
    #FLATTEN LIST OF NGRAM LISTS INTO LIST
    all_n_grams = [item for sublist in all_n_grams for item in sublist]
    
    #CONVERT TUPLES TO STR
    str_grams = []
    for n_gram in all_n_grams:
        str_grams.append(' '.join(n_gram))
        
    #ADD REVIEW TO LIST OF REVIEWS FOR BOW
    reviews_processed.append(str_grams)
    
  return reviews_processed

In [0]:
def save_X_Q43(pos, stemming, n_grams, progress_printing):
  start_time = time.time()
  reviews_processed = create_reviews_processed_Q43(reviews, pos, stemming, n_grams) 
  X = create_X(reviews_processed, progress_printing)

  print("--- time: %s seconds ---" % round((time.time() - start_time), 2), "\n")

  file_name = "X101_Q43.npy".format(pos, stemming, n_grams)
  np.save(file_name, X)

In [0]:
save_X_Q43(1, 0, 1, "no_progress_printing") # pos, stemming, n_grams

In [0]:
model = "SVM"
X = np.load('X101_Q43.npy')
Y = np.load('Y.npy')
folds = 10

results, accuracies, variance = cross_validate(model, X, Y, folds)
np.save('Q43_CV_results_101L', results)

In [0]:
av_accuracy = sum(accuracies)/folds
print("accuracies", accuracies)
print("average_accuracy: ", av_accuracy)
print("variance", variance)

***Applying an SVM with POS tags and without closed form words results in an accuracy of 0.831***

Sign test

In [0]:
# With closed-class words
Q42_CV_results_101L = np.load('Q42_CV_results_101L.npy')
# Without closed-class words
Q43_CV_results_101L = np.load('Q43_CV_results_101L.npy')
p_value = sign_test(Q42_CV_results_101L, Q43_CV_results_101L)

The p-value is 0.77, so the difference between the SVM with closed for words and without closed form words is not significant.

# (Q5) Discussion (max. 500 words). (5pts)

> Based on your experiments, what are the effective features and techniques in sentiment analysis? What information do different features encode?
Why is this important? What are the limitations of these features and techniques?
 


In this practical, sentiment analysis was performed on a corpus of movie reviews using two different methods: a symbolic classifyer, and a Bayesian classifyer. The symbolic classifyer assigns classes to documents by use of a sentiment lexicon, where each word in the document is looked up and counts are kept for positive/negative entries. It can be extended with magnitudes, where words with a 'strong' magnitude weigh more than words with a 'weak' magnitude - Theoretically, more information is taken into account, improving accuracy. In line with this hypothesis, enriching the classifyer with magnitudes slightly improved accuracy.

Better results were achieved through use of Naive Bayes (NB), which operates on a Bag of Words representation.

In its primitive form, NB is ineffective because of data sparsity: during testing, the algorithm is likely to encounter unseen words - the conditional probabilities of these words evaluate to zero, rendering resulting predictions unreliable. This explains the poor initial classification accuracy of 50%. Laplacian smoothing was used to account for this. This method deals with data sparsity by redistributing some of the probability mass from common events to rare ones. Applying this technique yields an accuracy of 81%.

Arguably, NB outperforms the symbolic classifyer as NB employs a larger feature space (individual words versus lexicon features). A finer-grained approach, NB is able to pick up on more specific facts in the documents, which might explain its better performance. 

Stemming can be applied before construction of the BoW. Stemming crudely normalizes words by getting rid of different inflections, reducing data sparsity. Applying stemming did not result in a significant difference in accuracy for NB. Possibly, different word inflections do not necessarily encode information relevant to sentiment analysis, and thus it makes sense that stemming does not affect classification outcomes.

A BoW can be extended by incorporating n-grams, re-introducing certain linguistic phenomena that were lost initially when only using unigram features. Theoretically, longer sequences of words encode certain facts specific to the data, allowing for more accurate modelling. Indeed, incorporating n-grams yields a moderate improvement in accuracy, with the combination of uni-grams, bi-grams and tri-grams giving an accuracy of 85%. Adding (higher-order) n-grams significantly expands the feature space. Not only is this computationally more demanding, it can also present problems in terms of data sparsity - generally, adding context is only appropriate if sufficient evidence is present. 

Words can be augmented with POS tags to deal with polysemy, helping the model to disentangle different meanings of words. In our case, this did not improve accuracy - arguably, different meanings of a word do not necessarily correspond to great differences in expressed sentiment. Alternatively, it's possible that our simplistic implementation of NB fails to capitalize on this information, whereas more sophisticated models would benefit more.

Finally, classification was attempted through use of a SVM, yielding similar performance to NB. Arguably, the simplicity of both the input data yield limited potential in terms of realizable information extraction. Nevertheless, more experimentation with features and parameter settings is required to make an honest comparison.



***P.S. For an overview (table) of the results, see the attached image in the submission.***

# Submission 


In [0]:
# Write your names and student numbers here:
# Ard Snijders #12854913
# Tim van Loenhout #10741577

**That's it!**

- Check if you answered all questions fully and correctly. 
- Download your completed notebook using `File -> Download .ipynb` 
- Also save your notebook as a Github Gist. Get it by choosing `File -> Save as Github Gist`.  Make sure that the gist has a secret link (not public).
- Check if your answers are all included in the file you submit (e.g. check the Github Gist URL)
- Submit your .ipynb file and link to the Github Gist via *Canvas*. One submission per group. 